In [1]:
import mysql.connector
import pandas as pd
import csv

In [5]:
db_connection = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="", 
  auth_plugin='mysql_native_password'
)
print(db_connection)

# creating database_cursor to perform SQL operation to run queries
db_cursor = db_connection.cursor(buffered=True)

db_cursor.execute("USE ratingApp")


In [ ]:
# executing cursor with execute method and pass SQL query
db_cursor.execute("CREATE DATABASE ratingApp")

# get list of all databases
db_cursor.execute("SHOW DATABASES")

# print all databases
for db in db_cursor:
    print(db)

In [ ]:
# create Artists table
db_cursor.execute("""CREATE TABLE ARTISTS (artist_id VARCHAR(50) PRIMARY KEY NOT NULL,  
                                          artist_fname VARCHAR(50), 
                                          artist_mname VARCHAR(50), 
                                          artist_lname VARCHAR(50),
                                          artist_genre VARCHAR(50))""")

insert_artists = (
    "INSERT INTO ARTISTS(artist_id, artist_fname, artist_mname, artist_lname, artist_genre) "
    "VALUES (%s, %s, %s, %s, %s)"
)

In [ ]:
# create Albums table
db_cursor.execute("""CREATE TABLE ALBUMS (album_id VARCHAR(50) PRIMARY KEY NOT NULL,  
                                          album_title VARCHAR(50), 
                                          album_releaseDate DATETIME,
                                          maker_id VARCHAR(50),
                                          album_genre VARCHAR(50),
                                          FOREIGN KEY(maker_id) REFERENCES ARTISTS(artist_id) 
                                          ON DELETE CASCADE
                                          ON UPDATE NO ACTION)""")

insert_albums = (
    "INSERT INTO ALBUM(album_id, album_title, album_releaseDate, album_genre) "
    "VALUES (%s, %s, %s, %s)"
)

In [ ]:
# create Songs table
db_cursor.execute("""CREATE TABLE SONGS (song_id VARCHAR(50) PRIMARY KEY NOT NULL,  
                                          song_duration FLOAT, 
                                          song_title VARCHAR(50),
                                          song_rating INTEGER,
                                          singer_id VARCHAR(50),
                                          inalbum_id VARCHAR(50),
                                          song_genre VARCHAR(50),
                                          FOREIGN KEY(singer_id) REFERENCES ARTISTS(artist_id) ON DELETE CASCADE
                                          ON UPDATE NO ACTION,
                                          FOREIGN KEY(inalbum_id) REFERENCES ALBUMS(album_id) ON DELETE CASCADE
                                          ON UPDATE NO ACTION)""")

insert_songs = (
    "INSERT INTO SONGS(song_id, song_title, song_duration, song_rating, singer_id, inalbum_id, song_genre) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s)"
)

In [7]:
# create Users table
db_cursor.execute("""CREATE TABLE USERS (user_id VARCHAR(50) PRIMARY KEY NOT NULL,  
                                          user_userName VARCHAR(50), 
                                          user_password VARCHAR(50) 
                                          )""")

insert_user = (
    "INSERT INTO USERS(user_id, user_userName, user_password) "
    "VALUES (%s, %s, %s)"
)

In [8]:
# create LikedSongs table
db_cursor.execute("""CREATE TABLE LikedSongs (song_id VARCHAR(50),  
                 user_id VARCHAR(50),
                 FOREIGN KEY(song_id) REFERENCES SONGS(song_id) ON DELETE CASCADE
                 ON UPDATE NO ACTION,
                 FOREIGN KEY(user_id) REFERENCES USERS(user_id) ON DELETE CASCADE
                 ON UPDATE NO ACTION)""")

insert_likedsong = (
    "INSERT INTO LikedSongs(song_id, user_id) "
    "VALUES (%s, %s)"
)

In [10]:
db_cursor.execute("""alter table LikedSongs add primary key (song_id, user_id)""")

In [11]:
# create LikedAlbums table
db_cursor.execute("""CREATE TABLE LikedAlbums (album_id VARCHAR(50),  
                 user_id VARCHAR(50),
                 FOREIGN KEY(album_id) REFERENCES ALBUMS(album_id) ON DELETE CASCADE
                 ON UPDATE NO ACTION,
                 FOREIGN KEY(user_id) REFERENCES USERS(user_id) ON DELETE CASCADE
                 ON UPDATE NO ACTION)""")

insert_likedalbum = (
    "INSERT INTO LikedAlbums(album_id, user_id) "
    "VALUES (%s, %s)"
)

In [12]:
db_cursor.execute("""alter table LikedAlbums add primary key (album_id, user_id)""")

In [13]:
def findFromSong(songtitle): 
    frommsong_query = """SELECT s.song_id, s.song_title, 'song' as type 
                      FROM SONGS s
                      WHERE s.song_title like '%""" + songtitle + """%' """
    db_cursor.execute(frommsong_query)
    records = db_cursor.fetchall()
    
    return records

In [14]:
def findFromAlbum(albumtitle): 
    fromalbum_query = """SELECT a.album_id, a.album_title, 'album' as type 
                      FROM ALBUMS a
                      WHERE a.album_title like '%""" + albumtitle + """%' """
    db_cursor.execute(fromalbum_query)
    records = db_cursor.fetchall()
    
    return records

In [15]:
def findFromArtist(name): 
    fromartist_query = """SELECT a.artist_id, a.artist_fname, 'artist' as type 
                      FROM ARTISTS a
                      WHERE a.artist_fname like '%""" + name + """%' 
                      OR a.artist_mname like '%""" + name + """%'
                      OR a.artist_lname like '%""" + name + """%' """
    db_cursor.execute(fromartist_query)
    records = db_cursor.fetchall()
    
    return records

In [16]:
def barSearch(songtitle, albumtitle, name):
    songs = findFromSong(songtitle)
    albums = findFromAlbum(albumtitle)
    artists = findFromArtist(name)
    all_results = songs + albums + artists
    
    return all_results

In [17]:
def findFromGenre(genreName):
    frommsonggenre_query = """SELECT s.song_id, s.song_title, 'song' as type 
                      FROM SONGS s
                      WHERE s.song_genre like '%""" + genreName + """%' """
    db_cursor.execute(frommsonggenre_query)
    recordssong = db_cursor.fetchall()
    
    fromalbumgenre_query = """SELECT a.album_id, a.album_title, 'album' as type 
                      FROM ALBUMS a
                      WHERE a.album_genre like '%""" + genreName + """%' """
    db_cursor.execute(fromalbumgenre_query)
    recordsalbum = db_cursor.fetchall()
    
    fromartistgenre_query = """SELECT a.artist_id, a.artist_fname, 'artist' as type 
                      FROM ARTISTS a
                      WHERE a.artist_genre like '%""" + genreName + """%'  """
    db_cursor.execute(fromartistgenre_query)
    recordsartist = db_cursor.fetchall()
    
    all_records = recordssong + recordsalbum + recordsartist
    
    return all_records

In [19]:
def getLikedSongs(user_id):
    fromlikedsong_query = """SELECT s.song_id, s.song_title
                            FROM SONGS s
                            join LikedSongs ls
                            ON s.song_id = ls.song_id
                            WHERE ls.user_id = user_id """
    db_cursor.execute(fromlikedsong_query)
    records = db_cursor.fetchall()
    
    return records

In [20]:
def getLikedAlbums(user_id):
    fromlikedalbums_query = """SELECT a.album_id, a.album_title
                            FROM ALBUMS a
                            join LikedAlbums la
                            ON a.album_id = la.album_id
                            WHERE la.user_id = user_id """
    db_cursor.execute(fromlikedalbums_query)
    records = db_cursor.fetchall()
    
    return records

In [21]:
def getTop10():
    toptensong_query = """SELECT s.song_id, min(s.song_title) as song_title
                            FROM SONGS S
                            join LikedSongs ls
                            on s.song_id = ls.song_id
                            GROUP BY ls.song_id
                            ORDER BY count(ls.user_id) DESC
                            limit 10 """
    
    db_cursor.execute(toptensong_query)
    records = db_cursor.fetchall()
    
    return records